In [79]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [80]:
# Multi-Agent Multilingual AI Chatbot with Google Gemini
# A comprehensive voice-enabled assistant with 5 specialized persona agents

# ============================================================================
# SECTION 1: SETUP AND DEPENDENCIES
# ============================================================================

# Install required packages (run this cell first)

!pip install -q google-generativeai
!pip install -q langchain langchain-core langchain-google-genai
!pip install -q langchain-text-splitters langchain-chroma chromadb
!pip install -q sentence-transformers langdetect pydub SpeechRecognition gtts
!pip install -q python-dotenv requests
!pip install -q langchain-community sentence-transformers


In [120]:
import os
os.environ["GOOGLE_API_KEY"] = "XYZ"

In [121]:
# Import core libraries
import os
import json
import time
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field
from enum import Enum

In [122]:
# LLM and agent frameworks
import google.generativeai as genai

# --- LangChain import compatibility (newer versions moved modules) ---
try:
    # Newer LangChain package layout
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    # Older layout fallback
    from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_google_genai import GoogleGenerativeAIEmbeddings

try:
    # Newer Chroma integration package
    from langchain_chroma import Chroma
except ImportError:
    # Older layout fallback
    from langchain.vectorstores import Chroma

try:
    # Newer Document location
    from langchain_core.documents import Document
except ImportError:
    # Older layout fallback
    from langchain.schema import Document

In [123]:
print("✅ All setup imports completed successfully")


✅ All setup imports completed successfully


In [124]:
# Language detection
from langdetect import detect, LangDetectException

# Voice processing (optional - requires audio setup)
try:
    import speech_recognition as sr
    from gtts import gTTS
    VOICE_AVAILABLE = True
except ImportError:
    VOICE_AVAILABLE = False
    print("Voice libraries not available. Text-only mode enabled.")

print("✓ All dependencies imported successfully")

✓ All dependencies imported successfully


In [125]:
# ============================================================================
# SECTION 2: CONFIGURATION AND API KEYS
# ============================================================================
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise RuntimeError(
        "GOOGLE_API_KEY is not set. "
        "Set it as an environment variable before running this code."
    )

genai.configure(api_key=GOOGLE_API_KEY)
print("✅ Google Gemini API configured")

✅ Google Gemini API configured


In [126]:
# Model configuration
GEMINI_MODEL = "models/gemini-2.0-flash-lite"  # ✅ for higher rate limits
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # ✅ optional: for clarity (HF)


In [119]:
available = [m.name for m in genai.list_models() if "generateContent" in m.supported_generation_methods]
print("Model available?", GEMINI_MODEL in available)
# Optional: print a few models
print("Some available models:", available[:20])


Model available? True
Some available models: ['models/gemini-2.5-flash', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-exp-1206', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'models/gemini-flash-latest']


In [127]:
# Supported languages (ISO 639-1 codes)
SUPPORTED_LANGUAGES = {
    'en': 'English', 'es': 'Spanish', 'fr': 'French', 'de': 'German',
    'it': 'Italian', 'pt': 'Portuguese', 'ru': 'Russian', 'zh': 'Chinese',
    'ja': 'Japanese', 'hi': 'Hindi', 'ar': 'Arabic', 'ko': 'Korean'
}


In [131]:
# Configuration object for all settings
@dataclass
class SystemConfig:
    """Central configuration for the entire system"""
    google_api_key: str = GOOGLE_API_KEY
    model_name: str = GEMINI_MODEL
    embedding_model: str = EMBEDDING_MODEL
    temperature: float = 0.7
    max_output_tokens: int = 2048
    vector_db_path: str = "./chroma_db"
    chunk_size: int = 1000
    chunk_overlap: int = 200
    top_k_retrieval: int = 3
    enable_voice: bool = VOICE_AVAILABLE


In [132]:
# ============================================================
# 🔧 FINAL MODEL SELECTION + SYSTEM BUILD (SINGLE SOURCE)
# ============================================================

# ✅ Choose low-quota-friendly model
GEMINI_MODEL = "models/gemini-2.0-flash-lite"

# Build system config ONCE
config = SystemConfig(model_name=GEMINI_MODEL)

print("✅ Using Gemini model:", config.model_name)

# Build router + persona agents ONCE
router = RouterAgent(config)

persona_agents = {
    persona_type: PersonaAgent(persona_type, config, rag_manager)
    for persona_type in PersonaType
}

print("✅ Router & persona agents initialized")


✅ Using Gemini model: models/gemini-2.0-flash-lite
✅ Router & persona agents initialized


In [134]:
# ============================================================================
# SECTION 3: LANGUAGE DETECTION AND UTILITIES
# ============================================================================

class LanguageDetector:
    """Detects user's language and maintains conversation context"""
    
    @staticmethod
    def detect_language(text: str) -> str:
        """
        Detect the language of input text
        Returns ISO 639-1 language code (e.g., 'en', 'es')
        """
        try:
            # Use langdetect library for detection
            lang_code = detect(text)
            # Return detected language if supported, otherwise default to English
            return lang_code if lang_code in SUPPORTED_LANGUAGES else 'en'
        except LangDetectException:
            # If detection fails, default to English
            return 'en'
    
    @staticmethod
    def get_language_name(lang_code: str) -> str:
        """Get full language name from code"""
        return SUPPORTED_LANGUAGES.get(lang_code, 'English')
    
    @staticmethod
    def format_prompt_with_language(prompt: str, language: str) -> str:
        """Add language instruction to any prompt"""
        lang_name = LanguageDetector.get_language_name(language)
        return f"{prompt}\n\nIMPORTANT: Respond in {lang_name}."

In [135]:
# Test language detection
test_texts = [
    ("Hello, how are you?", "en"),
    ("Hola, ¿cómo estás?", "es"),
    ("Bonjour, comment allez-vous?", "fr"),
    ("你好，你好吗？", "zh")
]

print("\n--- Language Detection Test ---")
for text, expected in test_texts:
    detected = LanguageDetector.detect_language(text)
    status = "✓" if detected == expected else "✗"
    print(f"{status} '{text[:30]}...' → {detected} ({LanguageDetector.get_language_name(detected)})")



--- Language Detection Test ---
✓ 'Hello, how are you?...' → en (English)
✓ 'Hola, ¿cómo estás?...' → es (Spanish)
✓ 'Bonjour, comment allez-vous?...' → fr (French)
✗ '你好，你好吗？...' → en (English)


In [136]:
# ============================================================================
# SECTION 4: PERSONA DEFINITIONS (5 Specialized Agents)
# ============================================================================

class PersonaType(Enum):
    FITNESS = "fitness"
    MENTAL_HEALTH = "mental_health"
    FINANCE = "finance"
    IMMIGRATION = "immigration"
    PARENTING = "parenting"

@dataclass
class PersonaConfig:
    name: str
    type: PersonaType
    system_prompt: str
    keywords: List[str]
    rag_corpus: List[str]
    tools: List[str]
    safety_rules: List[str]

# Define all 5 personas with their configurations
PERSONAS = {
    PersonaType.FITNESS: PersonaConfig(
        name="Fitness Coach",
        type=PersonaType.FITNESS,
        system_prompt="""You are an experienced and supportive fitness coach. You help users with:
        - Workout planning and exercise recommendations
        - Basic nutrition education and healthy eating habits
        - Habit building and motivation for fitness goals
        - Form and technique guidance

        You provide evidence-based advice and encourage sustainable, healthy practices.
        Always consider the user's fitness level and any limitations they mention.""",
        keywords=['workout', 'exercise', 'fitness', 'gym', 'training', 'muscle', 'cardio',
                  'nutrition', 'diet', 'protein', 'calories', 'weight loss', 'strength'],
        rag_corpus=[
            "Cardiovascular exercise improves heart health. Aim for 150 minutes of moderate-intensity or 75 minutes of vigorous-intensity aerobic activity per week.",
            "Strength training should be performed 2-3 times per week, targeting all major muscle groups. Allow 48 hours between sessions for muscle recovery.",
            "Proper form is crucial for preventing injuries. Start with lighter weights and focus on technique before increasing resistance.",
            "Protein intake of 0.8-1.0g per kg of body weight supports general health. Athletes may need 1.2-2.0g per kg for muscle building.",
            "Hydration is essential for performance. Drink water before, during, and after exercise. Aim for 8-10 glasses daily.",
            "Progressive overload - gradually increasing weight, reps, or intensity - is key to continued improvement.",
            "Rest and recovery are as important as training. Get 7-9 hours of sleep and take rest days to allow muscles to repair.",
            "Compound exercises like squats, deadlifts, and bench press work multiple muscle groups and are efficient for building strength."
        ],
        tools=['web_search'],
        safety_rules=[
            "Never diagnose medical conditions",
            "Recommend seeing a doctor before starting new intense programs",
            "Avoid extreme diet advice",
            "Emphasize gradual, sustainable changes"
        ]
    ),

    PersonaType.MENTAL_HEALTH: PersonaConfig(
        name="Mental Health Counsellor",
        type=PersonaType.MENTAL_HEALTH,
        system_prompt="""You are a supportive and empathetic mental health counsellor and motivator.

        CRITICAL: You are NOT a licensed therapist or mental health professional. You provide:
        - Emotional support and active listening
        - Basic mental health education and psychoeducation
        - Coping strategies and stress management techniques
        - Motivational support and encouragement

        You MUST:
        - Always include a disclaimer that you're not a replacement for professional help
        - Recognize signs of crisis (self-harm, suicidal ideation, severe distress)
        - Immediately provide crisis resources if user expresses crisis symptoms
        - Encourage professional help for ongoing mental health concerns

        Your tone is warm, non-judgmental, validating, and encouraging.""",
        keywords=['anxiety', 'depression', 'stress', 'therapy', 'mental health', 'panic', 'worry',
                  'sad', 'overwhelmed', 'burnout', 'confidence', 'motivation', 'sleep', 'crisis'],
        rag_corpus=[
            "Deep breathing (4-7-8 technique) can help activate the parasympathetic nervous system and reduce acute anxiety symptoms.",
            "Cognitive reframing involves identifying negative thought patterns and replacing them with more balanced, evidence-based alternatives.",
            "Regular physical activity has been shown to reduce symptoms of anxiety and depression by improving mood-regulating neurotransmitters.",
            "Sleep hygiene basics: consistent bedtime, limit screens 1 hour before sleep, cool/dark room, avoid caffeine late in the day.",
            "If someone is in immediate danger or considering self-harm, encourage contacting emergency services or local crisis hotlines immediately."
        ],
        tools=['therapist_finder', 'web_search'],
        safety_rules=[
            "Always include disclaimer: not a licensed therapist",
            "Provide crisis resources if self-harm/suicidal ideation appears",
            "Encourage professional help for serious issues",
            "Avoid diagnosing conditions",
            "Be empathetic and non-judgmental"
        ]
    ),

    PersonaType.FINANCE: PersonaConfig(
        name="Personal Finance Educator",
        type=PersonaType.FINANCE,
        system_prompt="""You are a practical personal finance educator. You help users with:
        - Budgeting and expense tracking
        - Debt payoff strategies
        - Saving and emergency funds
        - Basic investing concepts (general education, not personalized advice)
        - Credit score basics
        - Financial goal planning

        IMPORTANT BOUNDARIES:
        - You do NOT provide personalized financial or investment advice
        - You provide general education and examples only
        - Encourage consulting certified financial professionals for major decisions
        - Be clear about assumptions and risks""",
        keywords=['budget', 'saving', 'debt', 'credit', 'invest', '401k', 'ira', 'loan', 'interest',
                  'mortgage', 'emergency fund', 'retirement', 'stocks', 'index fund'],
        rag_corpus=[
            "A common budgeting framework is 50/30/20: 50% needs, 30% wants, 20% savings/debt repayment. Adjust based on income and goals.",
            "An emergency fund typically covers 3-6 months of essential expenses; start with a smaller goal like $500-$1,000.",
            "Debt payoff methods: avalanche (highest interest first) minimizes total interest; snowball (smallest balance first) builds momentum.",
            "Index funds provide broad diversification and typically have low expense ratios compared to actively managed funds.",
            "Credit score factors include payment history, utilization, length of credit history, new credit, and credit mix."
        ],
        tools=['web_search'],
        safety_rules=[
            "Do not provide personalized investment advice",
            "Explain risks and uncertainty",
            "Encourage professional advice for major decisions",
            "Avoid guaranteeing outcomes"
        ]
    ),

    PersonaType.IMMIGRATION: PersonaConfig(
        name="Immigration Guide",
        type=PersonaType.IMMIGRATION,
        system_prompt="""You are an immigration information guide. You help users understand:
        - High-level immigration pathways and terminology
        - Typical documentation requirements (general)
        - Process steps and timelines (general)
        - How to find official resources

        IMPORTANT:
        - You are NOT a lawyer and do NOT provide legal advice
        - Always recommend consulting qualified immigration attorneys for case-specific guidance
        - Encourage using official government resources (USCIS, etc.)""",
        keywords=['visa', 'immigration', 'green card', 'uscis', 'asylum', 'citizenship', 'petition',
                  'work permit', 'adjustment of status', 'consular processing', 'eb1', 'h1b', 'f1'],
        rag_corpus=[
            "Family-based immigration allows US citizens and permanent residents to petition for certain family members. Immediate relatives of US citizens have priority.",
            "Employment-based immigration has five preference categories (EB-1 through EB-5), each with specific eligibility requirements.",
            "Asylum may be granted to those who fear persecution in their home country based on race, religion, nationality, political opinion, or membership in a particular social group.",
            "USCIS (United States Citizenship and Immigration Services) is the primary agency handling immigration benefits and services.",
            "Processing times vary significantly by visa type and service center. Check official USCIS processing times for current estimates."
        ],
        tools=['web_search', 'official_docs_search'],
        safety_rules=[
            "ALWAYS clarify you provide general information, not legal advice",
            "Strongly encourage consulting immigration attorneys",
            "Explain complex processes in simple terms",
            "Direct users to official government resources",
            "Never make case-specific recommendations"
        ]
    ),

    PersonaType.PARENTING: PersonaConfig(
        name="Parenting Guide",
        type=PersonaType.PARENTING,
        system_prompt="""You are a supportive parenting guide who helps parents with childcare and child development questions.

        You provide:
        - Age-appropriate developmental guidance
        - Practical parenting strategies and tips
        - Educational activity suggestions
        - Behavior management approaches
        - General wellness and safety information

        IMPORTANT BOUNDARIES:
        - You do NOT provide medical diagnoses or treatment recommendations
        - Always recommend consulting pediatricians for health concerns
        - You support various parenting philosophies with evidence-based information
        - Your tone is supportive, practical, and non-judgmental

        You help parents feel confident and informed while respecting that every child and family is unique.""",
        keywords=['baby', 'child', 'parenting', 'kids', 'toddler', 'teenager', 'discipline',
                  'development', 'education', 'sleep', 'feeding', 'behavior', 'school'],
        rag_corpus=[
            "Newborns typically sleep 14-17 hours per day in short periods. By 6 months, many babies can sleep 6-8 hour stretches at night.",
            "Positive reinforcement is more effective than punishment for shaping behavior. Praise specific behaviors you want to encourage.",
            "Reading to children daily supports language development, literacy skills, and parent-child bonding. Start from infancy.",
            "Screen time recommendations: Under 18 months, avoid except video chatting. Ages 2-5, limit to 1 hour of quality content daily with parent co-viewing.",
            "Developmental milestones vary, but most children walk by 15 months, speak in 2-word phrases by 24 months, and can write their name by age 5.",
            "Consistent routines help children feel secure and develop self-regulation. Bedtime routines are especially important for sleep.",
            "Emotional validation helps children develop emotional intelligence. Acknowledge their feelings even when setting limits on behavior.",
            "Play is crucial for development. It builds creativity, problem-solving, social skills, and physical coordination."
        ],
        tools=['web_search', 'pediatric_info_search'],
        safety_rules=[
            "Never provide medical diagnoses",
            "Always recommend consulting pediatricians for health issues",
            "Be non-judgmental about parenting choices",
            "Provide evidence-based information",
            "Support parental confidence while emphasizing child safety"
        ]
    )
}

# --- CORRECTION #1: Preserve the SystemConfig reference so it can’t be overwritten later ---
system_config = config  # <-- keep this line

print("\n✓ All 5 persona configurations loaded:")
for persona_type, persona_cfg in PERSONAS.items():  # <-- IMPORTANT: persona_cfg, not config
    print(f"  - {persona_cfg.name} ({len(persona_cfg.rag_corpus)} knowledge documents)")




✓ All 5 persona configurations loaded:
  - Fitness Coach (8 knowledge documents)
  - Mental Health Counsellor (5 knowledge documents)
  - Personal Finance Educator (5 knowledge documents)
  - Immigration Guide (5 knowledge documents)
  - Parenting Guide (8 knowledge documents)


In [152]:
# ============================================================================
# SECTION 5: RAG SETUP - VECTOR STORES FOR EACH PERSONA
# ============================================================================

from langchain_community.embeddings import HuggingFaceEmbeddings  

class RAGManager:
    """Manages RAG pipelines for each persona"""

    def __init__(self, config: SystemConfig):
        self.config = config
        self.vector_stores: Dict[PersonaType, Any] = {}
        self.embeddings = None

        # ✅ Initialize OPEN-SOURCE embeddings model (no quota, free)
        try:
            self.embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
            print("✓ Open-source embeddings initialized (all-MiniLM-L6-v2)")
        except Exception as e:
            print(f"✗ Failed to initialize embeddings: {e}")

    def create_vector_store(self, persona_type: PersonaType, documents: List[str]) -> None:
        """Create a vector store for a specific persona"""
        if not self.embeddings:
            print(f"✗ Cannot create vector store for {persona_type.value} - embeddings not initialized")
            return

        try:
            # Convert strings to Document objects
            docs = [
                Document(page_content=doc, metadata={"source": f"{persona_type.value}_corpus"})
                for doc in documents
            ]

            # Create text splitter for chunking long documents
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.config.chunk_size,
                chunk_overlap=self.config.chunk_overlap,
                length_function=len
            )

            # Split documents into chunks
            split_docs = text_splitter.split_documents(docs)

            # Create vector store with Chroma
            vector_store = Chroma.from_documents(
                documents=split_docs,
                embedding=self.embeddings,
                collection_name=f"{persona_type.value}_collection",
                persist_directory=f"{self.config.vector_db_path}/{persona_type.value}"
            )

            self.vector_stores[persona_type] = vector_store
            print(f"✓ Vector store created for {persona_type.value} ({len(split_docs)} chunks)")

        except Exception as e:
            print(f"✗ Vector store creation failed for {persona_type.value}: {e}")

    def retrieve_context(self, persona_type: PersonaType, query: str, k: int = None) -> List[str]:
        """Retrieve relevant context from a persona's vector store"""
        if persona_type not in self.vector_stores:
            return []

        k = k or self.config.top_k_retrieval

        try:
            results = self.vector_stores[persona_type].similarity_search(query, k=k)
            return [doc.page_content for doc in results]
        except Exception as e:
            print(f"✗ Retrieval failed for {persona_type.value}: {e}")
            return []

    def initialize_all_personas(self) -> None:
        """Initialize vector stores for all defined personas"""
        print("\n--- Initializing RAG for all personas ---")
        for persona_type, persona_cfg in PERSONAS.items():
            self.create_vector_store(persona_type, persona_cfg.rag_corpus)


# ✅ IMPORTANT: ensure you are passing a real SystemConfig object
system_config = SystemConfig()   # <-- do this to avoid Notebook variable collisions

rag_manager = RAGManager(system_config)
rag_manager.initialize_all_personas()


✓ Open-source embeddings initialized (all-MiniLM-L6-v2)

--- Initializing RAG for all personas ---
✓ Vector store created for fitness (8 chunks)
✓ Vector store created for mental_health (5 chunks)
✓ Vector store created for finance (5 chunks)
✓ Vector store created for immigration (5 chunks)
✓ Vector store created for parenting (8 chunks)


In [130]:
print(rag_manager.retrieve_context(PersonaType.FITNESS, "How much cardio should I do per week?", k=2))
print(rag_manager.retrieve_context(PersonaType.FINANCE, "What is the 50/30/20 rule?", k=2))


['Cardiovascular exercise improves heart health. Aim for 150 minutes of moderate-intensity or 75 minutes of vigorous-intensity aerobic activity per week.', 'Cardiovascular exercise improves heart health. Aim for 150 minutes of moderate-intensity or 75 minutes of vigorous-intensity aerobic activity per week.']
['A common budgeting framework is 50/30/20: 50% needs, 30% wants, 20% savings/debt repayment. Adjust based on income and goals.', 'A common budgeting framework is 50/30/20: 50% needs, 30% wants, 20% savings/debt repayment. Adjust based on income and goals.']


In [137]:
# ============================================================================
# SECTION 6: TOOL DEFINITIONS
# ============================================================================

class ToolExecutor:
    """Executes various tools that agents can use"""
    
    @staticmethod
    def web_search(query: str, language: str = 'en') -> str:
        """
        Simulated web search tool
        In production, integrate with Google Search API, Serper, or similar
        
        Args:
            query: Search query
            language: Language for results
        
        Returns:
            Search results as formatted string
        """
        # This is a placeholder - in production, use actual search API
        # Example: Google Custom Search API, Serper API, or Tavily
        
        print(f"🔍 [Web Search] Query: '{query}' (language: {language})")
        
        # Simulated response - replace with actual API call
        simulated_results = f"""
        Search Results for: {query}
        
        1. [Result 1] Recent guidelines and information about {query}
        2. [Result 2] Expert recommendations on {query}
        3. [Result 3] Latest updates regarding {query}
        
        Note: This is a simulated search. Integrate with a real search API for production use.
        Recommended: Google Custom Search API (https://developers.google.com/custom-search)
        """
        
        return simulated_results.strip()
    
    @staticmethod
    def therapist_finder(location: str, specialty: str = "general") -> str:
        """
        Find therapists near a location
        In production, integrate with Psychology Today API or similar
        
        Args:
            location: City or area to search
            specialty: Type of therapy needed
        
        Returns:
            Formatted list of resources
        """
        print(f"🔍 [Therapist Finder] Location: {location}, Specialty: {specialty}")
        
        # Placeholder - replace with actual therapist directory API
        return f"""
        Mental Health Resources near {location}:
        
        1. Psychology Today Directory: psychologytoday.com/us/therapists
        2. SAMHSA Treatment Locator: findtreatment.gov or call 1-800-662-4357
        3. Better Help Online Therapy: betterhelp.com
        4. Crisis Resources:
           - National Suicide Prevention Lifeline: 988
           - Crisis Text Line: Text HOME to 741741
        
        Please verify credentials and ensure providers are licensed in your state.
        """
    
    @staticmethod
    def document_analyzer(document_text: str, analysis_type: str = "financial") -> Dict:
        """
        Analyze financial documents (simulated)
        In production, use OCR and parsing libraries
        
        Args:
            document_text: Text content of document
            analysis_type: Type of analysis to perform
        
        Returns:
            Dictionary with analysis results
        """
        print(f"📄 [Document Analyzer] Analyzing {analysis_type} document")
        
        # Simulated analysis - in production, parse actual document structure
        return {
            "type": analysis_type,
            "summary": "Document contains financial information including income and expenses",
            "key_figures": {
                "total_income": "Example: $5,000",
                "total_expenses": "Example: $3,500",
                "net": "Example: $1,500"
            },
            "recommendation": "This is educational analysis only, not financial advice. Consult a financial advisor for personalized guidance."
        }
    
    @staticmethod
    def official_docs_search(query: str, country: str = "US") -> str:
        """
        Search official government immigration resources
        
        Args:
            query: Search query
            country: Country whose immigration system to search
        
        Returns:
            Links to official resources
        """
        print(f"🔍 [Official Docs Search] Query: {query}, Country: {country}")
        
        if country.upper() == "US":
            return f"""
            Official US Immigration Resources for: {query}
            
            - USCIS Official Website: uscis.gov
            - Check Case Status: egov.uscis.gov/casestatus
            - Processing Times: egov.uscis.gov/processing-times
            - Forms Library: uscis.gov/forms
            - Policy Manual: uscis.gov/policy-manual
            
            For legal advice on your specific case, consult an immigration attorney.
            Find attorneys: ailalawyer.org (American Immigration Lawyers Association)
            """
        else:
            return f"Please consult official immigration resources for {country}."
    
    @staticmethod
    def pediatric_info_search(query: str) -> str:
        """
        Search pediatric health information from reputable sources
        
        Args:
            query: Health/development query
        
        Returns:
            Information from reputable pediatric sources
        """
        print(f"🔍 [Pediatric Info Search] Query: {query}")
        
        return f"""
        Pediatric Information for: {query}
        
        Reputable Sources:
        - American Academy of Pediatrics (AAP): healthychildren.org
        - CDC Child Development: cdc.gov/ncbddd/childdevelopment
        - KidsHealth: kidshealth.org
        
        IMPORTANT: This is general educational information only.
        For specific health concerns about your child, always consult your pediatrician.
        For emergencies, call 911 or go to the nearest emergency room.
        """

# Test tools
print("\n--- Tool Executor Test ---")
print(ToolExecutor.web_search("fitness workout routines"))



--- Tool Executor Test ---
🔍 [Web Search] Query: 'fitness workout routines' (language: en)
Search Results for: fitness workout routines

        1. [Result 1] Recent guidelines and information about fitness workout routines
        2. [Result 2] Expert recommendations on fitness workout routines
        3. [Result 3] Latest updates regarding fitness workout routines

        Note: This is a simulated search. Integrate with a real search API for production use.
        Recommended: Google Custom Search API (https://developers.google.com/custom-search)


In [138]:
print(ToolExecutor.therapist_finder("anxiety help"))


🔍 [Therapist Finder] Location: anxiety help, Specialty: general

        Mental Health Resources near anxiety help:

        1. Psychology Today Directory: psychologytoday.com/us/therapists
        2. SAMHSA Treatment Locator: findtreatment.gov or call 1-800-662-4357
        3. Better Help Online Therapy: betterhelp.com
        4. Crisis Resources:
           - National Suicide Prevention Lifeline: 988
           - Crisis Text Line: Text HOME to 741741

        Please verify credentials and ensure providers are licensed in your state.
        


In [75]:
# Find a Gemini model that supports generateContent for YOUR API key
models = list(genai.list_models())

candidates = []
for m in models:
    methods = getattr(m, "supported_generation_methods", []) or []
    if "generateContent" in methods:
        candidates.append(m.name)

print("✅ Models available for generateContent:")
for name in candidates[:30]:
    print(" -", name)

# Choose a safe default: prefer a "flash" model if available
chosen = None
for name in candidates:
    if "flash" in name.lower():
        chosen = name
        break
chosen = chosen or (candidates[0] if candidates else None)

if not chosen:
    raise RuntimeError("No available Gemini models support generateContent for this API key.")

print("\n✅ Using model:", chosen)

# Update the SystemConfig you are using
system_config.model_name = chosen


✅ Models available for generateContent:
 - models/gemini-2.5-flash
 - models/gemini-2.5-pro
 - models/gemini-2.0-flash-exp
 - models/gemini-2.0-flash
 - models/gemini-2.0-flash-001
 - models/gemini-2.0-flash-exp-image-generation
 - models/gemini-2.0-flash-lite-001
 - models/gemini-2.0-flash-lite
 - models/gemini-2.0-flash-lite-preview-02-05
 - models/gemini-2.0-flash-lite-preview
 - models/gemini-exp-1206
 - models/gemini-2.5-flash-preview-tts
 - models/gemini-2.5-pro-preview-tts
 - models/gemma-3-1b-it
 - models/gemma-3-4b-it
 - models/gemma-3-12b-it
 - models/gemma-3-27b-it
 - models/gemma-3n-e4b-it
 - models/gemma-3n-e2b-it
 - models/gemini-flash-latest
 - models/gemini-flash-lite-latest
 - models/gemini-pro-latest
 - models/gemini-2.5-flash-lite
 - models/gemini-2.5-flash-image-preview
 - models/gemini-2.5-flash-image
 - models/gemini-2.5-flash-preview-09-2025
 - models/gemini-2.5-flash-lite-preview-09-2025
 - models/gemini-3-pro-preview
 - models/gemini-3-flash-preview
 - models/g

In [139]:
# ============================================================================
# SECTION 7: AGENT DEFINITIONS 
# ============================================================================

class ConversationState:
    """Maintains state across a conversation"""
    
    def __init__(self):
        self.history: List[Dict[str, str]] = []
        self.detected_language: str = 'en'
        self.current_persona: Optional[PersonaType] = None
        self.context: Dict[str, Any] = {}
    
    def add_message(self, role: str, content: str):
        """Add a message to conversation history"""
        self.history.append({"role": role, "content": content})
    
    def get_history_string(self, last_n: int = 5) -> str:
        """Get formatted conversation history"""
        recent = self.history[-last_n:] if len(self.history) > last_n else self.history
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in recent])

class RouterAgent:
    """Routes user queries to the appropriate persona agent"""

    def __init__(self, config: SystemConfig, use_llm_routing: bool = True):
        self.config = config
        self.use_llm_routing = use_llm_routing
        self.model = genai.GenerativeModel(config.model_name)

    @staticmethod
    def _safe_response_text(response) -> str:
        """Extract text safely from Gemini response across SDK variants."""
        # 1) Try quick accessor
        try:
            txt = getattr(response, "text", None)
            if txt:
                return txt.strip()
        except Exception:
            pass

        # 2) Try candidates -> content -> parts -> text
        try:
            cands = getattr(response, "candidates", None)
            if cands:
                cand0 = cands[0]
                content = getattr(cand0, "content", None)
                parts = getattr(content, "parts", None) if content else None
                if parts:
                    out = "".join([getattr(p, "text", "") for p in parts]).strip()
                    if out:
                        return out
        except Exception:
            pass

        return ""

    def route_query(self, query: str, language: str) -> PersonaType:
        """Determine which persona should handle the query"""

        # If disabled (useful for demos to avoid extra Gemini calls)
        if not self.use_llm_routing:
            return self._keyword_based_routing(query)

        routing_prompt = f"""You are a query router. Analyze the user's query and determine which specialized assistant should handle it.

Available assistants:
1. FITNESS - workout plans, exercise, nutrition, fitness goals
2. MENTAL_HEALTH - emotional support, stress, anxiety, motivation, coping strategies
3. FINANCE - budgeting, saving, investing, financial education, money management
4. IMMIGRATION - visa, citizenship, immigration processes, work permits
5. PARENTING - childcare, child development, parenting advice, kids' health and education

User query: "{query}"

Respond with ONLY ONE WORD - the name of the assistant: FITNESS, MENTAL_HEALTH, FINANCE, IMMIGRATION, or PARENTING

Your response (one word only):"""

        try:
            response = self.model.generate_content(
                routing_prompt,
                generation_config=genai.GenerationConfig(
                    temperature=0.1,
                    max_output_tokens=10
                )
            )

            raw = self._safe_response_text(response).upper()

            # If Gemini returned no usable text (finish_reason=2 case), fallback cleanly
            if not raw:
                return self._keyword_based_routing(query)

            route = raw.split()[0]  # keep only the first token
            route_mapping = {
                'FITNESS': PersonaType.FITNESS,
                'MENTAL_HEALTH': PersonaType.MENTAL_HEALTH,
                'FINANCE': PersonaType.FINANCE,
                'IMMIGRATION': PersonaType.IMMIGRATION,
                'PARENTING': PersonaType.PARENTING
            }

            persona = route_mapping.get(route)
            if persona:
                print(f"✓ [Router] Query routed to: {PERSONAS[persona].name}")
                return persona

            return self._keyword_based_routing(query)

        except Exception as e:
            # If rate-limited or response is weird, fallback to keywords
            msg = str(e)
            if "429" in msg:
                return self._keyword_based_routing(query)

            print(f"✗ [Router] Error during routing: {e}")
            return self._keyword_based_routing(query)

    def _keyword_based_routing(self, query: str) -> PersonaType:
        """Fallback routing based on keyword matching"""
        query_lower = query.lower()

        scores = {persona_type: 0 for persona_type in PersonaType}
        for persona_type, persona_config in PERSONAS.items():
            for keyword in persona_config.keywords:
                if keyword in query_lower:
                    scores[persona_type] += 1

        best_persona = max(scores, key=scores.get)
        if scores[best_persona] > 0:
            print(f"✓ [Router] Keyword-based routing to: {PERSONAS[best_persona].name}")
            return best_persona

        print(f"⚠ [Router] No clear match, defaulting to Fitness Coach")
        return PersonaType.FITNESS


class PersonaAgent:
    """Individual persona agent that handles queries in its domain"""
    
    def __init__(self, persona_type: PersonaType, config: SystemConfig, rag_manager: RAGManager):
        self.persona_type = persona_type
        self.persona_config = PERSONAS[persona_type]
        self.config = config
        self.rag_manager = rag_manager
        self.model = genai.GenerativeModel(config.model_name)
        self.tool_executor = ToolExecutor()
    
    def process_query(self, query: str, language: str, conversation_state: ConversationState) -> str:
        """Process a query using this persona's knowledge and tools"""
        print(f"\n{'='*60}")
        print(f"[{self.persona_config.name}] Processing query in {LanguageDetector.get_language_name(language)}")
        print(f"{'='*60}")
        
        rag_context = self.rag_manager.retrieve_context(self.persona_type, query)
        context_str = "\n\n".join(rag_context) if rag_context else "No specific knowledge retrieved."
        
        print(f"📚 Retrieved {len(rag_context)} relevant knowledge chunks")
        
        tool_results = self._execute_tools_if_needed(query)
        
        full_prompt = self._build_prompt(
            query=query,
            language=language,
            rag_context=context_str,
            tool_results=tool_results,
            conversation_history=conversation_state.get_history_string()
        )
        
        try:
            response = self.model.generate_content(
                full_prompt,
                generation_config=genai.GenerationConfig(
                    temperature=self.config.temperature,
                    max_output_tokens=self.config.max_output_tokens
                )
            )
            
            answer = response.text
            answer = self._add_safety_disclaimers(answer)
            
            print(f"✓ [{self.persona_config.name}] Response generated ({len(answer)} chars)")
            return answer
            
        except Exception as e:
            error_msg = f"I apologize, but I encountered an error processing your request: {str(e)}"
            print(f"✗ [{self.persona_config.name}] Error: {e}")
            return error_msg
    
    def _execute_tools_if_needed(self, query: str) -> Dict[str, str]:
        """Determine if tools are needed and execute them"""
        tool_results = {}
        query_lower = query.lower()
        
        if any(word in query_lower for word in ['latest', 'recent', 'current', 'today', 'news', 'update']):
            if 'web_search' in self.persona_config.tools:
                print("🔧 Executing: web_search")
                tool_results['web_search'] = self.tool_executor.web_search(query)
        
        if self.persona_type == PersonaType.MENTAL_HEALTH:
            if any(word in query_lower for word in ['therapist', 'counselor', 'find help', 'professional']):
                print("🔧 Executing: therapist_finder")
                tool_results['therapist_finder'] = self.tool_executor.therapist_finder("user location")
        
        elif self.persona_type == PersonaType.FINANCE:
            if any(word in query_lower for word in ['document', 'statement', 'pay stub', 'analyze']):
                print("🔧 Executing: document_analyzer")
                tool_results['document_analyzer'] = str(self.tool_executor.document_analyzer("sample document"))
        
        elif self.persona_type == PersonaType.IMMIGRATION:
            if any(word in query_lower for word in ['official', 'uscis', 'government', 'form']):
                print("🔧 Executing: official_docs_search")
                tool_results['official_docs_search'] = self.tool_executor.official_docs_search(query)
        
        elif self.persona_type == PersonaType.PARENTING:
            if any(word in query_lower for word in ['health', 'doctor', 'pediatric', 'medical']):
                print("🔧 Executing: pediatric_info_search")
                tool_results['pediatric_info_search'] = self.tool_executor.pediatric_info_search(query)
        
        return tool_results
    
    def _build_prompt(self, query: str, language: str, rag_context: str, 
                      tool_results: Dict[str, str], conversation_history: str) -> str:
        """Build the complete prompt for the LLM"""
        
        lang_name = LanguageDetector.get_language_name(language)
        
        prompt_parts = [
            f"ROLE AND INSTRUCTIONS:",
            self.persona_config.system_prompt,
            "",
            f"SAFETY RULES:",
        ]
        
        for rule in self.persona_config.safety_rules:
            prompt_parts.append(f"- {rule}")
        
        prompt_parts.extend([
            "",
            f"KNOWLEDGE BASE (relevant to this query):",
            rag_context,
            ""
        ])
        
        if tool_results:
            prompt_parts.append("TOOL RESULTS:")
            for tool_name, result in tool_results.items():
                prompt_parts.append(f"\n[{tool_name}]:\n{result}\n")
            prompt_parts.append("")
        
        if conversation_history:
            prompt_parts.extend([
                "RECENT CONVERSATION HISTORY:",
                conversation_history,
                ""
            ])
        
        prompt_parts.extend([
            f"USER QUERY:",
            query,
            "",
            f"IMPORTANT: Respond in {lang_name}. Be helpful, clear, and follow all safety rules.",
            "",
            "YOUR RESPONSE:"
        ])
        
        return "\n".join(prompt_parts)
    
    def _add_safety_disclaimers(self, response: str) -> str:
        """Add appropriate disclaimers based on persona type"""
        
        disclaimers = []
        
        if self.persona_type == PersonaType.MENTAL_HEALTH:
            if not "not a replacement" in response.lower() and not "professional help" in response.lower():
                disclaimers.append(
                    "\n\n⚠️ Important: I'm an AI assistant, not a licensed mental health professional. "
                    "This information is for educational purposes. Please consult a qualified therapist for personalized care."
                )
        
        elif self.persona_type == PersonaType.FINANCE:
            if not "not financial advice" in response.lower():
                disclaimers.append(
                    "\n\n⚠️ Disclaimer: This is educational information only, not financial, investment, or tax advice. "
                    "Consult a licensed financial advisor for personalized recommendations."
                )
        
        elif self.persona_type == PersonaType.IMMIGRATION:
            if not "not legal advice" in response.lower():
                disclaimers.append(
                    "\n\n⚠️ Disclaimer: This is general information only, not legal advice. "
                    "Immigration law is complex and case-specific. Please consult a licensed immigration attorney."
                )
        
        elif self.persona_type == PersonaType.PARENTING:
            if any(word in response.lower() for word in ['health', 'sick', 'illness', 'medical', 'symptom']):
                disclaimers.append(
                    "\n\n⚠️ Important: This is general information only. "
                    "For health concerns about your child, always consult your pediatrician."
                )
        
        if self.persona_type == PersonaType.MENTAL_HEALTH:
            crisis_keywords = ['suicide', 'kill myself', 'self-harm', 'hurt myself', 'end it all']
            if any(word in response.lower() for word in crisis_keywords):
                disclaimers.insert(0,
                    "\n\n🚨 CRISIS RESOURCES - Available 24/7:\n"
                    "• National Suicide Prevention Lifeline: Call or text 988\n"
                    "• Crisis Text Line: Text HOME to 741741\n"
                    "• Emergency: Call 911\n"
                )
        
        return response + "".join(disclaimers)

# ----------------------------------------------------------------------------
# UPDATED INIT BLOCK (FIX): use a fresh SystemConfig instead of `config`
# ----------------------------------------------------------------------------
# In notebooks, `config` often gets overwritten (e.g., by persona loops), turning
# into a PersonaConfig. That breaks RouterAgent/PersonaAgent which expect SystemConfig.

system_config = SystemConfig()

router = RouterAgent(system_config)
persona_agents = {
    persona_type: PersonaAgent(persona_type, system_config, rag_manager)
    for persona_type in PersonaType
}

print("\n✓ Router and all 5 persona agents initialized")



✓ Router and all 5 persona agents initialized


In [140]:
# ============================
# Rebuild router + agents AFTER RouterAgent patch
# ============================

router = RouterAgent(config, use_llm_routing=True)

persona_agents = {
    persona_type: PersonaAgent(persona_type, config, rag_manager)
    for persona_type in PersonaType
}

orchestrator = MultiAgentOrchestrator(router, persona_agents)

print("✓ Rebuilt router, persona agents, and orchestrator")
print("✓ Using Gemini model:", config.model_name)


✓ Rebuilt router, persona agents, and orchestrator
✓ Using Gemini model: models/gemini-2.0-flash-lite


In [141]:
# ============================================================================
# SECTION 8: ORCHESTRATION - MAIN CONVERSATION LOOP
# ============================================================================

class MultiAgentOrchestrator:
    """Orchestrates the entire multi-agent system"""
    
    def __init__(self, router: RouterAgent, persona_agents: Dict[PersonaType, PersonaAgent]):
        self.router = router
        self.persona_agents = persona_agents
        self.language_detector = LanguageDetector()
    
    def process_user_input(self, user_input: str, conversation_state: ConversationState) -> str:
        """
        Main entry point - processes user input through the full pipeline
        
        Args:
            user_input: User's message (text or transcribed speech)
            conversation_state: Current conversation state
        
        Returns:
            Agent's response
        """
        print(f"\n{'*'*70}")
        print(f"USER INPUT: {user_input}")
        print(f"{'*'*70}")
        
        # Step 1: Detect language
        detected_lang = self.language_detector.detect_language(user_input)
        conversation_state.detected_language = detected_lang
        lang_name = self.language_detector.get_language_name(detected_lang)
        print(f"🌍 Detected language: {lang_name} ({detected_lang})")
        
        # Step 2: Route to appropriate persona
        persona_type = self.router.route_query(user_input, detected_lang)
        conversation_state.current_persona = persona_type
        
        # Step 3: Get the response from the selected persona agent
        persona_agent = self.persona_agents[persona_type]
        response = persona_agent.process_query(user_input, detected_lang, conversation_state)
        
        # Step 4: Update conversation history
        conversation_state.add_message("user", user_input)
        conversation_state.add_message("assistant", response)
        
        return response
    
    def run_interactive_session(self):
        """Run an interactive text-based conversation session"""
        print("\n" + "="*70)
        print("🤖 MULTI-AGENT AI ASSISTANT")
        print("="*70)
        print("Available personas: Fitness Coach, Mental Health Counsellor,")
        print("                    Finance Educator, Immigration Guide, Parenting Guide")
        print("\nType 'quit' or 'exit' to end the conversation")
        print("="*70 + "\n")
        
        conversation_state = ConversationState()
        
        while True:
            try:
                user_input = input("\nYou: ").strip()
                
                if not user_input:
                    continue
                
                if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
                    print("\n👋 Thank you for using the Multi-Agent AI Assistant. Goodbye!")
                    break
                
                # Process the input
                response = self.process_user_input(user_input, conversation_state)
                
                print(f"\n🤖 Assistant ({PERSONAS[conversation_state.current_persona].name}):")
                print("-" * 70)
                print(response)
                print("-" * 70)
                
            except KeyboardInterrupt:
                print("\n\n👋 Session ended by user. Goodbye!")
                break
            except Exception as e:
                print(f"\n❌ Error: {e}")
                continue

# Initialize orchestrator
orchestrator = MultiAgentOrchestrator(router, persona_agents)
print("\n✓ Multi-Agent Orchestrator initialized and ready")


✓ Multi-Agent Orchestrator initialized and ready


In [142]:
# ============================================================================
# TEST / RUN CHAT (MANUAL TEST)
# ============================================================================

# Initialize conversation state
conversation_state = ConversationState()

# Test query
user_query = "I feel anxious and can't sleep"
language = "en"

# Step 1: Route the query
persona = router.route_query(user_query, language)
print("Routed to persona:", persona)

# Step 2: Process the query with the selected persona
response = persona_agents[persona].process_query(
    user_query,
    language,
    conversation_state
)

# Step 3: Update conversation state
conversation_state.add_message("user", user_query)
conversation_state.add_message("assistant", response)

print("\n--- FINAL RESPONSE ---\n")
print(response)


✓ [Router] Keyword-based routing to: Mental Health Counsellor
Routed to persona: PersonaType.MENTAL_HEALTH

[Mental Health Counsellor] Processing query in English
📚 Retrieved 0 relevant knowledge chunks
✗ [Mental Health Counsellor] Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
Please retry in 44.964175413s. [links {
  description: "Learn more about Gemini 

In [104]:
!pip install -q SpeechRecognition gTTS


In [143]:
# ============================================================================
# SECTION 9: VOICE MODE (Cross-platform friendly for Streamlit)
# Requires only: SpeechRecognition, gTTS
# Install (you already did):
#   !pip install -q SpeechRecognition gTTS
#
# Notes:
# - In Streamlit/web, server-side microphone capture is NOT reliable.
#   Use browser audio recording/upload, then pass audio bytes/path into speech_to_text().
# - For audio playback in Streamlit: use mp3 bytes returned by text_to_speech_bytes().
#   Example: st.audio(mp3_bytes, format="audio/mp3")
# ============================================================================

import io
import os
import time
import tempfile
from typing import Optional

try:
    import speech_recognition as sr
    from gtts import gTTS
    VOICE_AVAILABLE = True
except Exception as e:
    VOICE_AVAILABLE = False
    _VOICE_IMPORT_ERROR = e


class VoiceInterface:
    """Handles voice input and output (Streamlit-friendly)."""

    def __init__(self, enabled: bool = VOICE_AVAILABLE):
        self.enabled = enabled and VOICE_AVAILABLE
        self.recognizer = None

        if not VOICE_AVAILABLE:
            print(f"ℹ️  Voice interface disabled (missing libs): {_VOICE_IMPORT_ERROR}")
            self.enabled = False
            return

        if self.enabled:
            self.recognizer = sr.Recognizer()
            print("✓ Voice interface initialized")
        else:
            print("ℹ️  Voice interface disabled")

    def _microphone_available(self) -> bool:
        """Check if microphone capture is possible in this runtime."""
        if not self.enabled:
            return False
        try:
            # This will fail if PyAudio / mic backend isn't available
            with sr.Microphone() as _:
                return True
        except Exception:
            return False

    def speech_to_text(
        self,
        audio_source: str = "microphone",
        language: str = "en",
        audio_bytes: Optional[bytes] = None,
    ) -> Optional[str]:
        """
        Convert speech to text.

        Args:
            audio_source:
              - 'microphone' (local notebooks; requires working microphone backend)
              - path to an audio file (wav/aiff/flac recommended; mp3 may not work everywhere)
              - 'bytes' (provide audio_bytes from Streamlit upload/recording; we save to temp file)
            language: ISO code, e.g., 'en'
            audio_bytes: audio content if audio_source == 'bytes'

        Returns:
            Transcribed text or None if failed
        """
        if not self.enabled:
            print("❌ Voice input not available (VoiceInterface disabled)")
            return None

        try:
            print("🎤 Processing audio...")

            if audio_source == "microphone":
                if not self._microphone_available():
                    print(
                        "❌ Microphone capture isn't available in this environment.\n"
                        "   For Streamlit/mobile, use browser recording/upload and pass audio as bytes/file path."
                    )
                    return None

                with sr.Microphone() as source:
                    # Adjust for ambient noise
                    self.recognizer.adjust_for_ambient_noise(source, duration=0.8)
                    audio = self.recognizer.listen(source, timeout=7, phrase_time_limit=12)

            elif audio_source == "bytes":
                if not audio_bytes:
                    print("❌ audio_bytes not provided")
                    return None

                # Streamlit often provides WAV/WEBM depending on browser.
                # SpeechRecognition works best with WAV/AIFF/FLAC. If you feed WEBM/MP3, it may fail.
                # We'll save bytes to a temp file and try sr.AudioFile (works for WAV/AIFF/FLAC).
                with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as f:
                    tmp_path = f.name
                    f.write(audio_bytes)

                try:
                    with sr.AudioFile(tmp_path) as source:
                        audio = self.recognizer.record(source)
                finally:
                    try:
                        os.remove(tmp_path)
                    except OSError:
                        pass

            else:
                # Assume it's a file path
                path = audio_source
                with sr.AudioFile(path) as source:
                    audio = self.recognizer.record(source)

            # Use Google Speech Recognition (free endpoint via SpeechRecognition)
            text = self.recognizer.recognize_google(audio, language=language)
            text = text.strip()
            print(f"✓ Transcribed: {text}")
            return text

        except sr.WaitTimeoutError:
            print("❌ No speech detected (timeout)")
            return None
        except sr.UnknownValueError:
            print("❌ Could not understand audio")
            return None
        except sr.RequestError as e:
            print(f"❌ Speech recognition request error: {e}")
            return None
        except Exception as e:
            print(f"❌ Speech recognition error: {e}")
            return None

    def text_to_speech_bytes(self, text: str, language: str = "en") -> Optional[bytes]:
        """
        Convert text to speech and return MP3 bytes.
        This is the best option for Streamlit: st.audio(mp3_bytes, format="audio/mp3")
        """
        if not self.enabled:
            print("❌ Voice output not available (VoiceInterface disabled)")
            return None

        try:
            tts = gTTS(text=text, lang=language, slow=False)
            buf = io.BytesIO()
            tts.write_to_fp(buf)
            return buf.getvalue()
        except Exception as e:
            print(f"❌ Text-to-speech error: {e}")
            return None

    def text_to_speech(self, text: str, language: str = "en", output_file: str = "response.mp3") -> Optional[str]:
        """
        Convert text to speech and save to an MP3 file (for local use).
        Returns the saved file path or None.
        """
        if not self.enabled:
            print("❌ Voice output not available (VoiceInterface disabled)")
            return None

        try:
            tts = gTTS(text=text, lang=language, slow=False)
            tts.save(output_file)
            print(f"✓ Audio saved to: {output_file}")
            return output_file
        except Exception as e:
            print(f"❌ Text-to-speech error: {e}")
            return None


# Initialize voice interface (does NOT break if mic isn't available)
voice_interface = VoiceInterface()


def voice_conversation_loop():
    """
    Voice-based conversation loop (local notebook/terminal only).
    For Streamlit/mobile: do NOT use this loop; instead:
      - collect audio via st.audio_input() or st.file_uploader()
      - call voice_interface.speech_to_text(audio_source="bytes", audio_bytes=...)
      - call orchestrator.process_user_input(...)
      - call voice_interface.text_to_speech_bytes(...) and st.audio(...)
    """
    if not voice_interface.enabled:
        print("❌ Voice features not available. Please install: pip install SpeechRecognition gTTS")
        return

    if not voice_interface._microphone_available():
        print(
            "❌ Microphone mode isn't available in this environment.\n"
            "   Use Streamlit browser recording/upload and the bytes/file path workflow instead."
        )
        return

    print("\n🎤 VOICE CONVERSATION MODE")
    print("=" * 70)
    print("Speak your question when prompted. Say 'stop' or 'exit' to end.")
    print("=" * 70)

    conversation_state = ConversationState()

    while True:
        user_text = voice_interface.speech_to_text(audio_source="microphone", language="en")
        if not user_text:
            continue

        if any(word in user_text.lower() for word in ["stop", "exit", "quit", "goodbye"]):
            print("👋 Ending voice conversation")
            break

        # Process through orchestrator (same as your original design)
        response = orchestrator.process_user_input(user_text, conversation_state)

        print(f"\n🤖 Response:\n{response}\n")

        # Save MP3 locally (for notebook). In Streamlit you should use text_to_speech_bytes instead.
        voice_interface.text_to_speech(
            response,
            language=conversation_state.detected_language or "en",
            output_file=f"response_{int(time.time())}.mp3"
        )


✓ Voice interface initialized


In [144]:
# ============================================================================
# SECTION 10: EXAMPLE DEMONSTRATIONS
# ============================================================================

import re
import time

def run_example_queries():
    """Quota-friendly demo run (reduces router LLM calls + slows down requests)."""

    print("\n" + "="*70)
    print("🎯 RUNNING EXAMPLE DEMONSTRATIONS")
    print("="*70)

    test_queries = [
        ("What's a good workout routine for beginners?", "en"),
        ("¿Cuánta proteína debo comer al día?", "es"),
        ("I've been feeling really anxious lately", "en"),
        ("Je me sens stressé au travail", "fr"),
        ("How does compound interest work?", "en"),
        ("Wie erstelle ich ein Budget?", "de"),
        ("What is a green card?", "en"),
        ("Come funziona il visto H1B?", "it"),
        ("My toddler won't sleep through the night", "en"),
        ("Quando deve começar a falar um bebê?", "pt"),
    ]

    conversation_state = ConversationState()

    # IMPORTANT: disable LLM routing during demo to halve Gemini calls
    router.use_llm_routing = False

    for i, (query, _) in enumerate(test_queries, 1):
        print(f"\n{'─'*70}")
        print(f"EXAMPLE {i}/{len(test_queries)}")
        print(f"{'─'*70}")

        try:
            response = orchestrator.process_user_input(query, conversation_state)
            print("\n📝 RESPONSE:")
            print(response[:900] + "..." if len(response) > 900 else response)

        except Exception as e:
            msg = str(e)

            # Auto-wait and retry once on 429
            if "429" in msg:
                m = re.search(r"Please retry in ([0-9]+(\.[0-9]+)?)s", msg)
                wait_s = float(m.group(1)) if m else 35.0
                wait_s = max(wait_s, 10.0)

                print(f"⏳ Rate-limited (429). Waiting {wait_s:.1f}s then retrying once...")
                time.sleep(wait_s)

                response = orchestrator.process_user_input(query, conversation_state)
                print("\n📝 RESPONSE (after retry):")
                print(response[:900] + "..." if len(response) > 900 else response)

            else:
                print(f"✗ Demo error: {e}")

        # Slow down to avoid per-minute free-tier limits
        time.sleep(12)

    router.use_llm_routing = True  # restore normal routing for real chats

    print("\n" + "="*70)
    print("✓ All example demonstrations completed!")
    print("="*70)


In [148]:
# ============================================================================
# SECTION 10B: EXAMPLE DEMONSTRATIONS (RAG-ONLY / NO LLM CALLS)
# ============================================================================

def run_example_queries_rag_only():
    """
    Run demonstration queries showing persona-specific RAG retrieval
    WITHOUT calling any LLM (no Gemini).
    """

    print("\n" + "="*70)
    print("🎯 RUNNING EXAMPLE DEMONSTRATIONS (RAG-ONLY MODE)")
    print("="*70)

    # Keep the same examples you already use (edit/add freely)
    example_queries = [
        (PersonaType.FITNESS, "What's a good workout routine for beginners?"),
        (PersonaType.FITNESS, "¿Cuánta proteína debo comer al día?"),
        (PersonaType.MENTAL_HEALTH, "I've been feeling really anxious lately"),
        (PersonaType.FINANCE, "How does compound interest work?"),
        (PersonaType.FINANCE, "Wie erstelle ich ein Budget?"),
        (PersonaType.IMMIGRATION, "What is a green card?"),
        (PersonaType.IMMIGRATION, "Come funziona il visto H1B?"),
        (PersonaType.PARENTING, "My toddler won't sleep through the night"),
        (PersonaType.PARENTING, "Quando deve começar a falar um bebê?"),
    ]

    for i, (persona, query) in enumerate(example_queries, start=1):
        print("\n" + "─"*70)
        print(f"EXAMPLE {i}/{len(example_queries)}")
        print("─"*70)

        print("\n" + "*"*70)
        print(f"USER INPUT: {query}")
        print("*"*70)

        # Retrieve only (no LLM generation)
        chunks = rag_manager.retrieve_context(persona, query, k=3)

        print(f"\nPersona: {PERSONAS[persona].name}")
        print(f"📚 Retrieved {len(chunks)} relevant knowledge chunks\n")

        if not chunks:
            print("⚠️ No RAG context found. (Vector store may be empty or not initialized.)")
        else:
            for idx, ch in enumerate(chunks, start=1):
                # Print a short preview for readability
                preview = ch.strip().replace("\n", " ")
                preview = preview[:500] + ("..." if len(preview) > 500 else "")
                print(f"[Chunk {idx}] {preview}\n")

    print("\n✅ Done. (RAG-only demo completed — no LLM calls were made.)")


In [157]:
# ============================================================================
# SECTION 11: MAIN EXECUTION 
# ============================================================================

def main():
    """Main execution function (Notebook-safe)."""

    # Basic sanity checks (prevents confusing NameErrors in notebooks)
    if "orchestrator" not in globals():
        print("❌ orchestrator is not defined yet. Run the orchestrator initialization cell first.")
        return

    print("\n" + "=" * 70)
    print("🚀 MULTI-AGENT AI ASSISTANT - READY")
    print("=" * 70)

    while True:
        print("\nChoose an option:")
        print("1. Run example demonstrations")
        print("2. Start interactive text conversation")
        print("3. Start voice conversation (if available)")
        print("4. Exit")

        choice = input("\nYour choice (1-4): ").strip()

        if choice == "1":
            if "run_example_queries" not in globals():
                print("❌ run_example_queries() is not defined yet. Paste/run Section 10 first.")
            else:
                run_example_queries()

        elif choice == "2":
            orchestrator.run_interactive_session()

        elif choice == "3":
            if "voice_conversation_loop" not in globals():
                print("❌ voice_conversation_loop() is not defined yet. Paste/run Section 9 first.")
            else:
                # Updated Section 9 voice loop expects orchestrator
                try:
                    voice_conversation_loop(orchestrator)
                except TypeError:
                    # Backward compatibility if your voice_conversation_loop takes no args
                    voice_conversation_loop()

        elif choice == "4":
            print("👋 Goodbye!")
            break

        else:
            print("Invalid choice. Please select 1-4.")


In [149]:
# ============================================================================
# SECTION 11B: MAIN EXECUTION (RAG-ONLY DEMO)
# ============================================================================

def main_rag_only():
    """
    Minimal main() for RAG-only demo.
    This never calls Gemini/LLM.
    """

    print("\n" + "="*70)
    print("🚀 MULTI-AGENT DEMO — RAG ONLY (NO LLM CALLS)")
    print("="*70)

    print("\nChoose an option:")
    print("1. Run RAG-only example demonstrations")
    print("2. Exit")

    choice = input("\nYour choice (1-2): ").strip()

    if choice == "1":
        run_example_queries_rag_only()
    elif choice == "2":
        print("👋 Goodbye!")
    else:
        print("Invalid choice. Please run main_rag_only() again and select 1-2.")


In [150]:
# ============================================================================
# SECTION 11C: DEMO MENU (BOTH MODES)
# ============================================================================

def main_demo_modes():
    """
    Demo launcher to show BOTH:
    1) RAG+LLM (your existing main / run_example_queries)
    2) RAG-only (new)
    """

    print("\n" + "="*70)
    print("🚀 MULTI-AGENT DEMO — CHOOSE MODE")
    print("="*70)

    print("\nChoose an option:")
    print("1. Run RAG+LLM example demonstrations (existing)")
    print("2. Run RAG-only example demonstrations (new, no LLM calls)")
    print("3. Run interactive chat (RAG+LLM) (existing)")
    print("4. Exit")

    choice = input("\nYour choice (1-4): ").strip()

    if choice == "1":
        # Uses your existing code
        run_example_queries()
    elif choice == "2":
        run_example_queries_rag_only()
    elif choice == "3":
        orchestrator.run_interactive_session()
    elif choice == "4":
        print("👋 Goodbye!")
    else:
        print("Invalid choice. Please run main_demo_modes() again and select 1-4.")


In [153]:
main_rag_only()



🚀 MULTI-AGENT DEMO — RAG ONLY (NO LLM CALLS)

Choose an option:
1. Run RAG-only example demonstrations
2. Exit



Your choice (1-2):  1



🎯 RUNNING EXAMPLE DEMONSTRATIONS (RAG-ONLY MODE)

──────────────────────────────────────────────────────────────────────
EXAMPLE 1/9
──────────────────────────────────────────────────────────────────────

**********************************************************************
USER INPUT: What's a good workout routine for beginners?
**********************************************************************

Persona: Fitness Coach
📚 Retrieved 3 relevant knowledge chunks

[Chunk 1] Compound exercises like squats, deadlifts, and bench press work multiple muscle groups and are efficient for building strength.

[Chunk 2] Compound exercises like squats, deadlifts, and bench press work multiple muscle groups and are efficient for building strength.

[Chunk 3] Compound exercises like squats, deadlifts, and bench press work multiple muscle groups and are efficient for building strength.


──────────────────────────────────────────────────────────────────────
EXAMPLE 2/9
─────────────────────────────

In [ ]:
main()



🚀 MULTI-AGENT AI ASSISTANT - READY

Choose an option:
1. Run example demonstrations
2. Start interactive text conversation
3. Start voice conversation (if available)
4. Exit



Your choice (1-4):  1



🎯 RUNNING EXAMPLE DEMONSTRATIONS

──────────────────────────────────────────────────────────────────────
EXAMPLE 1/10
──────────────────────────────────────────────────────────────────────

**********************************************************************
USER INPUT: What's a good workout routine for beginners?
**********************************************************************
🌍 Detected language: English (en)
✓ [Router] Keyword-based routing to: Fitness Coach

[Fitness Coach] Processing query in English
📚 Retrieved 0 relevant knowledge chunks
✓ [Fitness Coach] Response generated (4900 chars)

📝 RESPONSE:
Okay, let's build a great workout routine for you as a beginner! It's fantastic that you're starting, and I'm here to help you get off to a strong and safe start. Remember, the most important thing is consistency and listening to your body.

**Before we dive in, a quick reminder: If you have any underlying health conditions, or haven't exercised in a while, it's always a goo